In [1]:
import tensorflow as tf

from tensorflow.python.framework import graph_util

from keras.models import load_model, Model
from keras import backend

Using TensorFlow backend.


In [4]:
def extractgraph(model_path):
    session = tf.Session()
    backend.set_session(session)
    
    model = load_model(model_path)
    gd = session.graph.as_graph_def()
    
#     print([(n.name, [i for i in n.input]) for n in graph_def.node[:10]])
    
    for node in gd.node:
        if node.op == "Switch":
            node.op = "Identity"
            del node.input[1]
        
#     Print dropout nodes
#     print([(n.name, [i for i in n.input]) for n in graph_def.node if n.name.lower().find('dropout') != -1])
    
#     output_graph_def = graph_util.convert_variables_to_constants(session, graph_def, ['MatMul'])
    output_graph_def = gd
    
    tf.train.write_graph(gd, ".", "graph.pb", as_text=False)

#     with tf.gfile.GFile('graph.pb', "wb") as f:
#         f.write(output_graph_def.SerializeToString())
        
    return (model, output_graph_def)
    

In [72]:
# import os
# import sys

# version = 0
# model_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin/v' + str(version) + '/model.h5'

# (model, output_graph_def) = extractgraph(model_path)

# backend.clear_session

# session = tf.Session()

# backend.set_session(session)




[('a', []), ('b', []), ('MatMul', ['a', 'b']), ('a_1', []), ('b_1', []), ('MatMul_1', ['a_1', 'b_1']), ('a_2', []), ('b_2', []), ('MatMul_2', ['a_2', 'b_2']), ('a_3', [])]


In [14]:
def inspectGraph(graph_path, shouldPrint = False):
    g = tf.GraphDef()
    
    g.ParseFromString(open(graph_path, "rb").read())
    if shouldPrint: print(" ---| INPUTS: \n"), print([n for n in g.node if n.name.find("input") != -1])
    
    ops = set([n.op for n in g.node])
    if shouldPrint: print(" ---| OPS: \n"), print(" ".join(ops))

    